In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## Define folders

In [2]:
dir_ELUC_NGHGI = '/work/mj0060/m300896/GCB2021_commentary/Data/data_ELUC_NGHGI/'
dir_SLAND      = '/work/mj0060/m300896/GCB2021_commentary/Data/data_ELUC_NGHGI_SLAND_plot/'
dir_ctrs       = '/work/mj0060/m300896/Trendy/Data/data_ancillary/info_countries/'
dir_ELUC_2021  = '/work/mj0060/m300896/GCB2021_commentary/Data/ELUC_countries/'
dir_peat       = '/work/mj0060/m300896/GCB2021/Data/Peat_data/'
dir_out        = '/work/mj0060/m300896/GCB2021_commentary/Data/repository_data/'


## Read country names and define countries for output

In [3]:
fname_cntrs = dir_ctrs + 'Country codes 3 letters.xlsx'
data_cntrs  = pd.read_excel(fname_cntrs, sheet_name=0, header=None, index_col=0)

#Select countries
countries_sel = ['USA', 'RUS', 'CAN', 'CHN', 'BRA', 'IDN', 'COD']
countries_EU  = ['AUT', 'BEL', 'BGR', 'HRV', 'CYP', 'CZE', 'DNK', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HUN', 'IRL', 'ITA', 'LVA', 'LTU', 'LUX', 'MLT', 'NLD', 'POL', 'PRT', 'ROU', 'SVK', 'SVN', 'ESP', 'SWE', 'GBR']


## Define time

In [4]:
#Define time
time_sta = 2001
time_end = 2015
time_len = time_end - time_sta + 1
time_str = str(time_sta) + '-' + str(time_end)

print(time_str)


2001-2015


## Read peat data

In [5]:
#Read peat data
fname_peat_drai = dir_peat + 'Country_emissions_peat-drainage_FAO_1850-2021.xlsx'
fname_peat_fire = dir_peat + 'Country_emissions_peat-fires_GFED4_1997-2021.xlsx'
data_peat_in_1 = pd.read_excel(fname_peat_drai, header=0, index_col=0)
data_peat_in_2 = pd.read_excel(fname_peat_fire, header=0, index_col=0)
data_peat_in_1 = data_peat_in_1.loc[(data_peat_in_1.index>=time_sta) & (data_peat_in_1.index<=time_end)]
data_peat_in_2 = data_peat_in_2.loc[(data_peat_in_2.index>=time_sta) & (data_peat_in_2.index<=time_end)]

#Select time
if (data_peat_in_1.index[0]!=time_sta) | (data_peat_in_1.index[-1]!=time_end) | (len(data_peat_in_1.index)!=time_len):  sys.exit('Check time selection of peat 1!')
if (data_peat_in_2.index[0]!=time_sta) | (data_peat_in_2.index[-1]!=time_end) | (len(data_peat_in_2.index)!=time_len):  sys.exit('Check time selection of peat 2!')


## Prepare BLUE data (and add peat)

In [6]:
#Read ELUC data
fname_ELUC  = dir_ELUC_2021 + 'ELUC_BLUE_countries-ISOcode_ELUC-net_vRemapCountries_2000-2020.xlsx'
data_BLUE_read = pd.read_excel(fname_ELUC, header=0, index_col=0)

#Select time
data_BLUE_read = data_BLUE_read.loc[(data_BLUE_read.index>=time_sta) & (data_BLUE_read.index<=time_end)]

#Check time selection
if (data_BLUE_read.index[0]!=time_sta) | (data_BLUE_read.index[-1]!=time_end) | (len(data_BLUE_read.index)!=time_len):  sys.exit('Check time selection of BLUE!')

#Create empty dataframe
data_ELUC_BLUE_2021_all = data_BLUE_read.copy()
data_ELUC_BLUE_2021_all[:] = np.NaN

#Loop over all countries
for col in data_BLUE_read.columns:
    
    #Sum ELUC and peat
    if col in data_peat_in_1.columns:
        data_BLUE_read[col] = data_BLUE_read[col] + data_peat_in_1[col]
    if col in data_peat_in_2.columns:
        data_BLUE_read[col] = data_BLUE_read[col] + data_peat_in_2[col]

    #Save in dataframe
    data_ELUC_BLUE_2021_all[col] = data_BLUE_read[col]

#Add data from Taiwan and Hongkong to China
data_ELUC_BLUE_2021_all['CHN'] = data_ELUC_BLUE_2021_all['CHN'] + data_ELUC_BLUE_2021_all['TWN'] + data_ELUC_BLUE_2021_all['HKG']    
    
#Select countries
data_BLUE_ctrs = data_ELUC_BLUE_2021_all[countries_sel]

#Calculate sum over EU countries and add to dataframe
data_BLUE_EU   = data_ELUC_BLUE_2021_all[countries_EU].sum(axis=1)
data_BLUE_ctrs = data_BLUE_ctrs.assign(EU27_UK=data_BLUE_EU)

## UNITS
#data_ELUC_BLUE_2021 has units: Tg C / year


## Prepare H&N2021 data

In [7]:
#Read ELUC data (which already include peat)
fname_ELUC = dir_ELUC_NGHGI + 'HN2021_ELUC-net-with-peat_GCB2021_countries.xlsx'
data_HN2021_read = pd.read_excel(fname_ELUC, header=0, index_col=0)

#Select time
data_HN2021_read = data_HN2021_read.loc[(data_HN2021_read.index>=time_sta) & (data_HN2021_read.index<=time_end)]

#Check time selection
if (data_HN2021_read.index[0]!=time_sta) | (data_HN2021_read.index[-1]!=time_end) | (len(data_HN2021_read.index)!=time_len):  sys.exit('Check time selection of H&N2021!')

#Create dataframe for storing data
data_HN2021_ctrs = pd.DataFrame(index=data_HN2021_read.index)

#Loop over selected countries
for country in countries_sel:

    #Get long name for country
    ctr_long = data_cntrs.loc[country].item()
    
    #Change country name
    if "Congo, Democratic Republic of the" in ctr_long:
        ctr_long = "Democratic Republic of the Congo"

    #Collect data in dataframe
    data_HN2021_ctrs[country] = data_HN2021_read[ctr_long]
    
#Loop over EU countries
data_HN2021_EU = pd.DataFrame(index=data_HN2021_read.index)
for country in countries_EU:

    #Get long name for country
    ctr_long = data_cntrs.loc[country].item()
    
    #Change country name
    if "United Kingdom of Great Britain and Northern Ireland" in ctr_long:
        ctr_long = "United Kingdom"
    elif "Czechia" in ctr_long:
        ctr_long = "Czech Republic"
    
    #Collect data in dataframe
    data_HN2021_EU[country] = data_HN2021_read[ctr_long]

#Calculate sum over EU countries and add to dataframe
data_HN2021_EU = data_HN2021_EU.sum(axis=1)
data_HN2021_ctrs = data_HN2021_ctrs.assign(EU27_UK=data_HN2021_EU)
      
## UNITS
#data_ELUC_HN_2021 has units: Tg C / year


## Prepare OSCAR data

In [8]:
#Read ELUC data 
fname_ELUC = dir_ELUC_NGHGI + 'OSCAR_ELUC-net-with-peat_GCB2021_IPCCcountries.xlsx'
data_OSCAR_read = pd.read_excel(fname_ELUC, header=0, index_col=0)

#Select time
data_OSCAR_read = data_OSCAR_read.loc[(data_OSCAR_read.index>=time_sta) & (data_OSCAR_read.index<=time_end)]

#Check time selection
if (data_OSCAR_read.index[0]!=time_sta) | (data_OSCAR_read.index[-1]!=time_end) | (len(data_OSCAR_read.index)!=time_len):  sys.exit('Check time selection of BLUE!')

#Select countries
data_OSCAR_ctrs = data_OSCAR_read[countries_sel]

#Calculate sum over EU countries and add to dataframe
data_OSCAR_EU   = data_OSCAR_read[countries_EU].sum(axis=1)
data_OSCAR_ctrs = data_OSCAR_ctrs.assign(EU27_UK=data_OSCAR_EU)

## UNITS
#data_ELUC_OSCAR_2021 has units: Tg C / year


## Save bookkeeping data in excel file

In [9]:
BM_models = ['BLUE', 'OSCAR', 'H&N2021']

#Define output file name
fname_out = dir_out + 'Fig2_Data_country-level_anthropogenic-LULUCF-flux_BM-models_' + time_str + '.xlsx'
if os.path.exists(fname_out): os.remove(fname_out)
    
#Create xlsx-file (it will be filled at end of loop with country data from every model)
with pd.ExcelWriter(fname_out) as writer:
    
    for model in BM_models:

        if model=='BLUE':      data = data_BLUE_ctrs
        elif model=='OSCAR':   data = data_OSCAR_ctrs
        elif model=='H&N2021': data = data_HN2021_ctrs
        
        #Convert Tg C/year to Tg CO2/year
        data = data * 44 / 12        

        #Add units in first cell and sort columns
        data = data.rename_axis('unit: Tg CO2/year')
        data = data.sort_index(axis=1)

        #Create sheet in xlsx for every model and store country data
        data.to_excel(writer, sheet_name=model, index=True, header=True, float_format='%.2f')


## Prepare SLAND data

In [10]:
#Read SLAND
file_name  = dir_SLAND + 'Collection_SLAND_total-weighted_' + time_str + '.pickle'
data_SLAND = pd.read_pickle(file_name)

#Read SLAND on forests
file_name = dir_SLAND + 'Collection_SLAND_non-intact-forest_mask2013_' + time_str + '.pickle'
data_SLAND_forest = pd.read_pickle(file_name)

#Convert unit from Tg C/year to Tg CO2/year
data_SLAND        = 44/12 * data_SLAND
data_SLAND_forest = 44/12 * data_SLAND_forest

#Select SLAND median for specific countries
countries_sel = ['BRA', 'CAN', 'CHN', 'COD', 'IDN', 'RUS', 'USA', 'EU27_UK']
data_SLAND_all_sel = data_SLAND.loc[countries_sel].median(axis=1)
data_SLAND_for_sel = data_SLAND_forest.loc[countries_sel].median(axis=1)

#Get uncertainty
models_SLAND = data_SLAND_forest.columns
data_SLAND_all_unc = data_SLAND.loc[countries_sel][models_SLAND]
data_SLAND_for_unc = data_SLAND_forest.loc[countries_sel][models_SLAND]

#Save uncertainty in data frames
data_DGVMs_ALL = pd.DataFrame(index=data_SLAND_all_unc.index, columns=['median', 'lower range', 'upper range'])
data_DGVMs_FOR = pd.DataFrame(index=data_SLAND_for_unc.index, columns=['median', 'lower range', 'upper range'])
data_DGVMs_ALL['median']      = data_SLAND_all_unc.quantile(0.50, axis=1)
data_DGVMs_ALL['lower range'] = data_SLAND_all_unc.quantile(0.25, axis=1)
data_DGVMs_ALL['upper range'] = data_SLAND_all_unc.quantile(0.75, axis=1)
data_DGVMs_FOR['median']      = data_SLAND_for_unc.quantile(0.50, axis=1)
data_DGVMs_FOR['lower range'] = data_SLAND_for_unc.quantile(0.25, axis=1)
data_DGVMs_FOR['upper range'] = data_SLAND_for_unc.quantile(0.75, axis=1)

data_SLAND_out = dict()
data_SLAND_out['all-land']        = data_DGVMs_ALL
data_SLAND_out['managed-forests'] = data_DGVMs_FOR

# ## UNITS
# data_DGVMs_ALL and data_DGVMs_FOR have units: Tg CO2 / year


## Save SLAND data in excel file

In [11]:
#Define data that should be stored
selections = ['all-land', 'managed-forests']

#Loop over selections
for selection in selections:
    
    #Define output file name and sheet names
    if selection=='all-land':
        fname_out = dir_out + 'Fig1_Data_country-level_natural-land-flux_DGVMs_' + time_str + '.xlsx'
        sheet_name = 'Total natural sink (all land)'
    elif selection=='managed-forests':
        fname_out = dir_out + 'Fig2_Data_country-level_natural-land-flux_DGVMs_' + time_str + '.xlsx'
        sheet_name = 'Natural sink in managed forests'
    
    #Select data
    data = data_SLAND_out[selection].sort_index()

    #Add units in first cell and sort
    data = data.rename_axis('unit: Tg CO2/year')
    data = data.sort_index()

    #Create sheet in xlsx for every model and store country data
    if os.path.exists(fname_out): os.remove(fname_out)
    data.to_excel(fname_out, sheet_name=sheet_name, index=True, header=True, float_format='%.2f')


## Read data NGHGI

In [12]:
#Read data   
fname_NGHGI = dir_ELUC_NGHGI + 'Grassi_Giacomo_NGHGI_2021_11_preliminary.xlsx'
data_NGHGI  = pd.read_excel(fname_NGHGI, sheet_name='summary DB2', header=2, index_col=0)

#Read new data for Indonesia
fname_IDN = dir_ELUC_NGHGI + 'LULUCF_Indonesia_UNFCCC_BUR3.xlsx'
data_IDN  = pd.read_excel(fname_IDN, header=0, index_col=0)

#Get column and index names
data_NGHGI.columns = data_NGHGI.iloc[0]
data_NGHGI.index = data_NGHGI['ISO']

#Select data
data_NGHGI = data_NGHGI.iloc[1:196, 12:16]
data_NGHGI = data_NGHGI.astype(float)



######################## Temporarily change data for China ########################

data_China = pd.DataFrame(columns=['NGHGI'], index=np.arange(2000, 2021))
data_China.loc[2005] =  -803000 / 1000
data_China.loc[2010] = -1029720 / 1000
data_China.loc[2012] =  -575848 / 1000
data_China.loc[2014] = -1150910 / 1000
data_China = data_China.astype(float)

#Interpolate and extrapolate data
data_China = data_China.interpolate(limit_direction='both')

#Add data to NGHGI
data_NGHGI.loc['CHN'][0] = data_China[(data_China.index>=2001) & (data_China.index<=2005)].mean()
data_NGHGI.loc['CHN'][1] = data_China[(data_China.index>=2006) & (data_China.index<=2010)].mean()
data_NGHGI.loc['CHN'][2] = data_China[(data_China.index>=2011) & (data_China.index<=2015)].mean()
data_NGHGI.loc['CHN'][3] = data_China[(data_China.index>=2016) & (data_China.index<=2020)].mean()

######################## Temporarily change data for China ########################



#Define index for time selection (start)
if time_sta==2001:  i_sta = 0
else:               sys.exit('Start time is not well defined for NGHGI')

#Define index for time selection (end)
if time_end==2010:    i_end = 2
elif time_end==2015:  i_end = 3
elif time_end==2020:  i_end = 4
else:                 sys.exit('End time is not well defined for NGHGI')

#Select time and calculate average
data_NGHGI = data_NGHGI.iloc[:, i_sta:i_end].mean(axis=1)
data_NGHGI = pd.Series(data_NGHGI, index = data_NGHGI.index).to_frame(name='NGHGI')

#Calculate average for Indonesia in selected period and add replace old NGHGI data of IDN
data_IDN  = data_IDN[data_IDN.columns[(data_IDN.columns>=time_sta) & (data_IDN.columns<=time_end)]].mean(axis=1)
data_NGHGI.loc['IDN'] = data_IDN['LULUCF flux'] / 1000

#Convert from Mt CO2 yr-1 to Mt C yr-1 (=Tg C / year)
data_NGHGI = 12 / 44 * data_NGHGI


## Add REDD+ data for DR Congo

#Read data   
fname_REDD = dir_ELUC_NGHGI + 'REDDplus_DRC.xlsx'
data_REDD  = pd.read_excel(fname_REDD, header=1, index_col=1)

#Select data and time period
data_REDD = data_REDD.iloc[0:3,1::]
data_REDD = data_REDD.loc['Net']
data_REDD = data_REDD[(data_REDD.index>=time_sta) & (data_REDD.index<=time_end)].mean()

#Convert from kt CO2 yr-1 to Mt C yr-1 (=Tg C / year)
data_REDD = 12 / 44 / 1000 * data_REDD

#Add to NGHGI data
data_COD = [data_REDD, data_NGHGI.loc['COD', 'NGHGI']]
data_NGHGI.loc['COD', 'NGHGI'] = np.mean(data_COD)

#Calculate sum over EU countries and add to dataframe
data_NGHGI_EU   = data_NGHGI.loc[countries_EU].sum()
data_NGHGI.loc['EU27_UK'] = data_NGHGI_EU

#Select countries
data_NGHGI_ctrs = data_NGHGI.loc[countries_sel]

#Rename column
data_NGHGI_ctrs = data_NGHGI_ctrs.rename(columns={'NGHGI': 'mean'})

#Add uncertainty
data_unc_perc = pd.DataFrame(columns=['uncertainty_lower', 'uncertainty_upper'])
data_unc_perc.loc['USA']     = [-17.1, 19.7]
data_unc_perc.loc['RUS']     = [-42.8, 42.8]
data_unc_perc.loc['CAN']     = [-209.0, 209.0]
data_unc_perc.loc['CHN']     = [-23.4, 23.4]
data_unc_perc.loc['BRA']     = [-34.9, 34.9]
data_unc_perc.loc['IDN']     = [-20.1, 20.1]
data_unc_perc.loc['EU27_UK'] = [-34.2, 34.2]
data_unc_perc = data_unc_perc.astype(float)

#Calculate uncertainty in Tg C /year
data_unc = pd.DataFrame(columns=['unc1', 'unc2'])
data_unc['unc1'] = data_NGHGI_ctrs['mean']  * (1 + data_unc_perc['uncertainty_upper'] / 100)
data_unc['unc2'] = data_NGHGI_ctrs['mean']  * (1 + data_unc_perc['uncertainty_lower'] / 100)

#Add uncertainty for COD
data_unc.loc['COD'] = [np.min(data_COD), np.max(data_COD)]

#Get minimum and maximum and drop unnecessary columns
data_unc['uncertainty_lower'] = data_unc.min(axis=1)
data_unc['uncertainty_upper'] = data_unc.max(axis=1)
data_unc = data_unc.drop(columns=['unc1', 'unc2'])

#Add uncertainty to data frame
data_NGHGI_ctrs = pd.concat((data_NGHGI_ctrs, data_unc), axis=1)

## UNITS
#data_NGHGI has units: Tg C / year


## Save NGHGI data in excel file

In [13]:
#Define output file name
fname_out = dir_out + 'Fig2_Data_country-level_LULUCF-flux_UNFCCC-reports_' + time_str + '-average.xlsx'
if os.path.exists(fname_out): os.remove(fname_out)
    
#Create xlsx-file (it will be filled at end of loop with country data from every model)
with pd.ExcelWriter(fname_out) as writer:

    #Convert Tg C/year to Tg CO2/year
    data = data_NGHGI_ctrs * 44 / 12        

    #Add units in first cell and sort
    data = data.rename_axis('unit: Tg CO2/year')
    data = data.sort_index()
    
    #Create sheet in xlsx for every model and store country data
    data.to_excel(writer, sheet_name='LULUCF from UNFCCC reports', index=True, header=True, float_format='%.2f')
